In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
import pandas as pd
import joblib
import re
import string
import snowballstemmer
import spacy
from nltk.tokenize import word_tokenize
import nltk


#import user dataset
path='/content/drive/MyDrive/18Gazete.csv' 
try:
  df = pd.read_csv(path)
except: 
  df = pd.read_csv(path, encoding = "utf-16")  

In [83]:
column_name = 'HABERMETNI'
content = df[column_name]

In [84]:
content[0]

"Hentbol ligi aslında bu sezon pek çok zorluklar ile bir araya gelebildi. Fon üretilememesi, maliyetlerin artması, spor camiasından desteğin sınırlı alması, yönetim hataları nedeni ile iki yılda 57 kulüp hentboldan çekildi. Erkeklerde Selka gibi çok değerli bir yapı terk etti hentbolu. Batman ve Adıyaman Belediyeleri kapattı. Yozgat Bozok çıkmayı reddedip Başkent Akademi'ye devretti takımı. Bu kadar zorluklar ile uğraşan hentbol camiası, İstanbul Belediyesi'ne ait olan ve bulunduğu bölgede hentbol ölçülerine sahip tek tesisi, Ümraniye Haldun Alagaş Spor Salonu'nu da kaybetti."

In [85]:
#Cleaning Process
pd.options.mode.chained_assignment = None  # default='warn'

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

i=0
for text in content:
  content[i]=clean_text(text)
  i=i+1

In [86]:
# Get N-number and Stopwords
stopwords_txt = "acaba,ama,aslında,az,bazı,belki,biri,birkaç,birşey,biz,bu,çok,çünkü,da,daha,de,defa,diye,eğer,en,gibi,hem,hep,hepsi,her,hiç,için,ile,ise,kez,ki,kim,mı,mu,mü,nasıl,ne,neden,nerde,nerede,nereye,niçin,niye,o,sanki,şey,siz,şu,tüm,ve,veya,ya,yani"
initial_stopwords = stopwords_txt.split(",")

#user's stopwords 
stopwords = []
a=False #if user does not give stopwords this variable = false
if a: 
  #kullanıcıdan gelen stopwordsleri split ile ayırıp for döngüsünde append ile stopwords listesine ekliyoruz.
  form="bir,ben,sen,o,şu,bu" #example
  arr = form.split(',')
  for i in arr:
    stopwords.append(i)
else:
  for text in initial_stopwords:
    stopwords.append(text)


#number of n
min_n_gram = 2
max_n_gram = 2 # user input

In [87]:
import nltk
nltk.download('punkt')
def stemmer_fun_stop(sentence,stopwords):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
      if word not in stopwords:
        stem_sentence.append(word)
        stem_sentence.append(" ") 
    return "".join(stem_sentence)

i=0
for text in content:
  content[i] = stemmer_fun_stop(text,stopwords)
  i=i+1  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [88]:
#!pip install zeyrek
import zeyrek
import nltk
analyzer = zeyrek.MorphAnalyzer()

def lemmatization(texts, allowed_postags=["Noun", 'Adj', 'Verb', 'Adv']):
      texts_out = []
      text = texts.split(" ")
      for sent in text:
        if sent !="":
          x=analyzer.analyze(sent)[0][0]
          if (x.pos=="Unk"):
            texts_out.append(analyzer.lemmatize(sent)[0][1][0])
          else:
            texts_out.append(x.lemma)
      return texts_out

data_lemmatized=[]
for text in content:
  data_lemmatized.append(lemmatization(text, allowed_postags=['Noun', 'Adj', 'Verb', 'Adv']))




In [89]:
data = []
def combine(txt):
  temp = []
  for word in txt:
    temp.append(word)
    temp.append(" ")
  return "".join(temp)

for txt in data_lemmatized:
  data.append(combine(txt)) 

In [90]:
# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=1000, ngram_range=(min_n_gram,max_n_gram)) 
tfidf = tfidf_vectorizer.fit_transform(data)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
# Run NMF
nmf = NMF(n_components=20, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

In [91]:
# To display words with desc. order 
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)     

Topic 0:
flâş haber dakika flâş son dakika terör örgüt suri sınır ilçe bir serbest bırakmak mahkeme tutuk park hal iki kişi
Topic 1:
ak parti parti sözcü ömer çelik sözcü ömer faruk çelik cumhurbaşkanı ak devam etmek açıklama bulunmak parti genel tartışmak ilgili
Topic 2:
avrupa lig uefa avrupa lig grup hafta maç grup ikinci ikinci hafta grup hafta şampiyon lig beşiktaş uefa borussia mönchengladbach
Topic 3:
savunmak bakan millî savunmak bakan hulusi hulusi akmak güven bölge abd savunmak akmak abd bakan akmak bölge tesis mark esper
Topic 4:
yazı sozcucomtrde gün yüz şüphe tutuk gerekmek kendi genel başkan genel kurul genel müdür genel sekreter genelkurmay başkan getirmek toplam
Topic 5:
hayat kaybetmek kişi hayat kişi yaralamak kaybetmek kişi sonuç kişi dünya savaş paris emniyet form giymek savaş kalmak asker yaralamak
Topic 6:
son dakika dakika haber haber göre orta çıkmak son durum yılmak önce haber stanbul fatih ter miss turkey hava etki
Topic 7:
suudi arabistan cemal kaşık gazete c

In [92]:
#Sample 
sample = """ 
  operasyon gözaltı alınmak gözaltı alınmak şüphe elemek uyuşturmak geçirildi elemek geçirildi düzenl
"""

sample_clean = clean_text(sample)
sample_lem = stemmer_fun_stop(sample_clean,stopwords)
sample_all_clean = lemmatization(sample_lem, allowed_postags=['Noun', 'Adj', 'Verb', 'Adv'])
sample_all_clean = combine(sample_all_clean)

# Transform the TF-IDF
test = tfidf_vectorizer.transform([sample_all_clean])
#  Transform the TF-IDF: nmf_features
nmf_features = nmf.transform(test)
 
 
def display_topics_of_sample(model, feature_names, no_top_words, topic_names , prct):
  y=19
  for x in range(5):
    topic_name = topic_names[0][y]
    y = y-1  
    for topic_idx, topic in enumerate(model.components_):
      if topic_name == topic_idx:
        print ("Topic percentage %" , prct[0][topic_name])
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

prct = nmf.transform(test)*100

#make list
def make_list(model, feature_names, no_top_words, topic_names , prct):
    lst = []
    y=19
    for x in range(5):
        topic_name = topic_names[0][y]
        y = y-1  
        for topic_idx, topic in enumerate(model.components_):
            if topic_name == topic_idx:
                lst.append( [prct[0][topic_name], " ".join([feature_names[i]
                                for i in topic.argsort()[:-no_top_words - 1:-1]])])
    return lst
    
no_top_words=10
prct = nmf.transform(test)*100
List = make_list(nmf, tfidf_feature_names, no_top_words, nmf.transform(test).argsort(axis=1), prct)

print(List)

[[28.736822218349424, 'gözaltı alınmak elemek geçirildi düzenlemek operasyon şüphe gözaltı kişi gözaltı şube müdür mücadele şube suç mücadele müdür ekip yönelik operasyon'], [0.0, 'avrupa birlik birlik ab gümrük vergi vergi uygulamak ab komisyon abd avrupa başkan jeanclaude jeanclaude juncker komisyon başkan yasmak dış'], [0.0, 'etki hal hal getirmek pkk terörist terörist etki hava harekât düzenlemek hava irakın kuzey bölge düzenlemek pençe harekât harekât pkk'], [0.0, 'ak parti parti sözcü ömer çelik sözcü ömer faruk çelik cumhurbaşkanı ak devam etmek açıklama bulunmak parti genel tartışmak ilgili'], [0.0, 'avrupa lig uefa avrupa lig grup hafta maç grup ikinci ikinci hafta grup hafta şampiyon lig beşiktaş uefa borussia mönchengladbach']]
